In [1]:
import pandas as pd
import os
from thefuzz import fuzz, process

In [2]:
player_folder_21_22 = "./data/2021-22/players/"
player_folder_22_23 = "./data/2022-23/players/"
player_folder_23_24 = "./data/2023-24/players/"
understat_player_folder_21_22 = "./data/2021-22/understat/"
understat_player_folder_22_23 = "./data/2022-23/understat/"
understat_player_folder_22_24 = "./data/2023-24/understat/"

subfolders_21_22 = [ f.path for f in os.scandir(player_folder_21_22) if f.is_dir() ]
subfolders_22_23 = [ f.path for f in os.scandir(player_folder_22_23) if f.is_dir() ]
subfolders_23_24 = [ f.path for f in os.scandir(player_folder_23_24) if f.is_dir() ]

files_21_22_under = [ f.path for f in os.scandir(understat_player_folder_21_22) if f.is_file() ]
files_22_23_under = [ f.path for f in os.scandir(understat_player_folder_22_23) if f.is_file() ]
files_23_24_under = [ f.path for f in os.scandir(understat_player_folder_22_24) if f.is_file() ]

In [3]:
# FPL folders
def save_player_name(subfolders, season):
    for folder in subfolders:
        player_name = folder.split('/')[4].split('_')
        clean_player_name = " ".join(player_name[0:-1])
        player_df = pd.read_csv(str(folder+'/gw.csv'))

        player_df.to_csv('./data/joint/'+str(season)+'/' + str(clean_player_name) + '.csv', index_label=False)

# Understat Files
def save_player_name_understat(files, season):
    for file_ in files_21_22_under:
        player_name = file_.split('/')[4].split('_')
        clean_player_name = " ".join(player_name[0:-1])
        player_df = pd.read_csv(str(file_))
        player_df.to_csv('./data/joint/'+str(season)+'/' + str(clean_player_name) + '.csv', index_label=False)

In [4]:
save_player_name(subfolders_21_22, '21-22/fpl')
save_player_name(subfolders_22_23, '22-23/fpl')
save_player_name(subfolders_23_24, '23-24/fpl')

save_player_name_understat(files_21_22_under, '21-22/understat')
save_player_name_understat(files_22_23_under, '22-23/understat')
save_player_name_understat(files_23_24_under, '23-24/understat')

In [5]:
player_ids_21_22 = pd.read_csv('./data/2021-22/id_dict.csv')
player_ids_21_22

,Understat_ID,FPL_ID,Understat_Name,FPL_Name
0,62,317,Lukas Rupp,Lukas Rupp
1,65,134,Timo Werner,Timo Werner
2,76,305,Fabian Schär,Fabian Schär
3,87,310,Joelinton,Joelinton Cássio Apolinário de Lira
4,101,307,Allan Saint-Maximin,Allan Saint-Maximin
...,...,...,...,...
604,10405,82,Josh Dasilva,Pelenda Joshua Dasilva
605,10408,696,Luis Díaz,Luis Díaz
606,10424,710,Daniel Chesters,Daniel Chesters
607,10525,716,Tony Springett,Tony Springett


In [148]:
player_ids_21_22.shape[0]

609

In [97]:
understat_21_22_path = "./data/joint/21-22/understat/"
fpl_player_21_22_path = "./data/joint/21-22/fpl/"
understat_22_23_path = "./data/joint/22-23/understat/"
fpl_player_22_23_path = "./data/joint/22-23/fpl/"
understat_23_24_path = "./data/joint/23-24/understat/"
fpl_player_23_24_path = "./data/joint/23-24/fpl/"

understat_21_22 = next(os.walk(understat_21_22_path), (None, None, []))[2]  # [] if no file
fpl_21_22 = next(os.walk(fpl_player_21_22_path), (None, None, []))[2]  # [] if no file

In [84]:
understat_21_22_names = [file_.split('.')[0] for file_ in understat_21_22]
fpl_21_22_names = [file_.split('.')[0] for file_ in fpl_21_22]

In [88]:
player_ids_21_22.index[player_ids_21_22[' Understat_Name'] == name].tolist()

[272]

In [154]:


x = None
y = None
m = None
n = None

for name in understat_21_22_names:
    # print(name)
    player_index = player_ids_21_22.index[player_ids_21_22[' Understat_Name'] == name].tolist()

    if(player_index):
        fpl_player_name = player_ids_21_22.loc[player_index[0], ' FPL_Name']

        if(fpl_player_name):
            # Use Fuzzy matching get the corresponding fpl name
            fuzzy_fpl_player_name = process.extractOne(fpl_player_name, fpl_21_22_names, scorer=fuzz.partial_token_sort_ratio)
            print(fuzzy_fpl_player_name)

            fpl_player_data = pd.read_csv(fpl_player_21_22_path + str(fuzzy_fpl_player_name[0])+ '.csv')
            understat_player_data = pd.read_csv(understat_21_22_path + name + '.csv')

            # Change 'kickoff_time' column name to 'date
            fpl_player_data = fpl_player_data.rename(columns={'kickoff_time': 'date'})
            # change the formats: From 2021-10-03T13:00:00Z to 2021-10-03
            fpl_player_data.date = fpl_player_data.date.apply(lambda x: x.split('T')[0])

            # Dates are of the form 2021-10-03T13:00:00Z
            fpl_dates_min = fpl_player_data['date'].min()
            fpl_dates_max = fpl_player_data['date'].max()
            print(fpl_dates_min, fpl_dates_max)

            # Filter out player info not in the range of dates we are dealing with
            understat_filtered = understat_player_data[(pd.to_datetime(understat_player_data['date']) > pd.to_datetime(fpl_dates_min))
                                                        & (pd.to_datetime(understat_player_data['date']) < pd.to_datetime(fpl_dates_max) )]

            # Marge fpl_player_data with understat_player_data if the dates match
            player_data_merged = fpl_player_data.merge(understat_filtered, on="date")
            if(player_data_merged.shape[0]):
                player_data_merged.to_csv('./data/joint/21-22/merged/'+ fpl_player_name +'.csv' )

('Aaron Connolly', 100)
2021-08-14 2022-05-22
('Aaron Cresswell', 100)
2021-08-15 2022-05-22
('Aaron Lennon', 100)
2021-08-29 2022-05-22
('Aaron Ramsdale', 100)
2021-08-22 2022-05-22
('Aaron Wan-Bissaka', 100)
2021-08-14 2022-05-22
('Abdoulaye Doucouré', 100)
2021-08-14 2022-05-22
('Adam Armstrong', 100)
2021-08-14 2022-05-22
('Adam Forshaw', 100)
2021-08-14 2022-05-22
('Adam Idah', 100)
2021-08-14 2022-05-22
('Adam Lallana', 100)
2021-08-14 2022-05-22
('Adam Masina', 100)
2021-08-14 2022-05-22
('Adam Webster', 100)
2021-08-14 2022-05-22
('Adama Traoré', 100)
2021-08-14 2022-05-22
('Ademola Lookman', 100)
2021-09-11 2022-05-22
('Adrián San Miguel del Castillo', 100)
2021-08-14 2022-05-22
('Ainsley Maitland-Niles', 100)
2021-08-13 2022-05-22
('Albert Sambi Lokonga', 100)
2021-08-13 2022-05-22
('Alejandro Garnacho Ferreyra', 100)
2022-04-16 2022-05-22
('Alex Iwobi', 100)
2021-08-14 2022-05-22
('Alex Kral', 100)
2021-09-11 2022-05-22
('Alex McCarthy', 100)
2021-08-14 2022-05-22
('Alex Oxl

In [139]:
n.date

0    2022-04-23
1    2022-04-28
2    2022-05-02
3    2022-05-07
4    2022-05-22
Name: date, dtype: object

In [145]:
m.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [81]:
x[(pd.to_datetime(understat_player_data['date']) >= pd.to_datetime( m)) & (pd.to_datetime(understat_player_data['date']) <= pd.to_datetime(n) )]


,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,id,season,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup


In [83]:
pd.read_csv('./data/joint/21-22/merged/Álvaro Fernández.csv')

,Unnamed: 0,assists_x,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,...,id,season,roster_id,xA,assists_y,key_passes,npg,npxG,xGChain,xGBuildup


In [45]:
pd.to_datetime('2022-05-22') == pd.to_datetime('2022-05-22')

True